In [3]:
import numpy as np

def parse_file(filename):
    """
    Parses the input file into header information, frequencies, magnitude sums, and matrices.
    """
    with open(filename, 'r') as f:
        lines = f.readlines()
        
    # Extract the number of frequencies and tracked points
    n_freqs = int(lines[0].strip())
    n_features = int(lines[1].strip())
    
    # Extract frequencies and magnitude sums
    freqs = []
    x_mag_sums = []
    y_mag_sums = []
    for i in range(2, 2 + n_freqs):
        freq, x_sum, y_sum = map(float, lines[i].strip().split(','))
        freqs.append(freq)
        x_mag_sums.append(x_sum)
        y_mag_sums.append(y_sum)
    
    # Extract the matrices
    matrix_lines = lines[2 + n_freqs:]
    matrices = []
    for i in range(4):  # xmag_real, xmag_imag, ymag_real, ymag_imag
        matrix = np.array([list(map(float, row.split(','))) for row in matrix_lines[i * n_freqs: (i + 1) * n_freqs]])
        matrices.append(matrix)
    
    return n_freqs, n_features, np.array(freqs), np.array(x_mag_sums), np.array(y_mag_sums), matrices

def normalize_matrix(matrix):
    """
    Normalizes a matrix by its maximum absolute value.
    """
    max_val = np.max(np.abs(matrix))
    return matrix / max_val if max_val != 0 else matrix

def process_files(file1, file2, output_file, weight1=0.6, weight2=0.4):
    """
    Processes two files, computes a weighted average of their normalized values, and saves the result.
    
    Parameters:
    - file1, file2: Input file paths.
    - output_file: Path for the output file.
    - weight1, weight2: Weights for the weighted average.
    """
    # Ensure weights sum to 1
    assert abs(weight1 + weight2 - 1.0) < 1e-6, "Weights must sum to 1!"
    
    # Parse both files
    data1 = parse_file(file1)
    data2 = parse_file(file2)
    
    # Ensure consistent structure
    assert data1[:2] == data2[:2], "File structures do not match!"
    n_freqs, n_features = data1[:2]
    freqs = data1[2]
    
    # Weighted average of frequencies and magnitude sums
    weighted_x_mag_sums = (weight1 * data1[3] + weight2 * data2[3])
    weighted_y_mag_sums = (weight1 * data1[4] + weight2 * data2[4])
    
    # Weighted average of normalized matrices
    weighted_matrices = []
    for mat1, mat2 in zip(data1[5], data2[5]):
        norm_mat1 = normalize_matrix(mat1)
        norm_mat2 = normalize_matrix(mat2)
        weighted_matrices.append(weight1 * norm_mat1 + weight2 * norm_mat2)
    
    # Write the weighted averaged data to a new file
    with open(output_file, 'w') as f:
        # Write header
        f.write(f"{n_freqs}\n")
        f.write(f"{n_features}\n")
        
        # Write frequencies and weighted magnitude sums
        for freq, x_sum, y_sum in zip(freqs, weighted_x_mag_sums, weighted_y_mag_sums):
            f.write(f"{freq},{x_sum},{y_sum}\n")
        
        # Write matrices
        for matrix in weighted_matrices:
            for row in matrix:
                f.write(",".join(map(str, row)) + "\n")



In [4]:
file1 = "../Lukas Kanade + Simulation/tmp/goodflower-fft_berthy.txt"
file2 = "../Lukas Kanade + Simulation/tmp/goodflowerlk-fft.txt"
output_file = "goodflower-fft.txt"

process_files(file1, file2, output_file, weight1=0.6, weight2=0.4)

